# Mining testimonial fragments of the Holocaust

**Experience domain: sit**

### Load the necessary libraries

In [406]:
import sys; sys.path.insert(0, '..')
import itertools

In [407]:
import get_topic_model_concordance as topic_concordancer
from utils import blacklab, db, text
mongo = db.get_db()

In [408]:
%config Completer.use_jedi = False
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import random

### Helper functions

In [409]:
def create_contextual_query(lemmas,context_length=50):
    permutations = itertools.permutations(lemmas,len(lemmas))
    final_result = []
    for element in list(permutations):
        temp_result = []
        for el in element:
            temp_result.append('[lemma="'+el+'"]')
        temp_result = '('+('[]{0,'+str(context_length)+'}').join(temp_result)+')'
        final_result.append(temp_result)
    final_result = '|'.join(final_result)
    return final_result
        
        
            

In [410]:
from utils import blacklab, db, text
import requests
import json
def find_sentence_id(label):
    props = {'annotators': 'tokenize'}

    # set the encoding of the annotator
    requests.encoding = 'utf-8'
    # make a request
    r = requests.post('http://localhost:9000/', params={'properties':
                      json.dumps(props)},
                      data=label.encode('utf-8'))
    result = json.loads(r.text, encoding='utf-8')
    query = []
    for i, token in enumerate(result['tokens']):

        if ('...'in token['word'] and ((i == 0) or
           i == len(result['tokens']) - 1)):
            continue
        elif ('...'in token['word']):
            query.append('[]{0,50}')
        elif ('-'in token['word']):
            query.append('[]{0,3}')
        elif ("n't"in token['word']):
            query.append('[]')
        elif ("'re"in token['word']):
            query.append('[]')
        elif ("?"in token['word']):
            query.append('[]')
        elif ("."in token['word']):
            query.append('[]')
        elif ("'s"in token['word']):
            query.append('[]')
        elif (","in token['word']):
            query.append('[]')
        else:
            query.append('["' + token['word'] + '"]')

    query = ' '.join(query)
    try:
        sentence = blacklab.search_blacklab(query, window=0,
                                            lemma=False,
                                            include_match=True)
        token_end = sentence[0]['token_end']
        token_start = sentence[0]['token_start']
        print (sentence[0])
        mongo = db.get_db()
        results = mongo.tokens.find({'testimony_id':
                                    sentence[0]['testimony_id']},
                                    {'_id': 0})
        tokens = list(results)[0]['tokens']
        sentenceStart = tokens[token_start]['sentence_index']
        sentenceEnd = tokens[token_end]['sentence_index']
        originalsentence = sentence[0]['complete_match']
        return (sentenceStart,sentenceEnd,sentence[0]['testimony_id'])
    except:
        print("The following query returned a null result")
        print(query)
        
            


In [411]:
def create_parent_node(label):
    """Generate a root node for a tree structure."""
    testimony_id = random.randint(1, 20)
    node = {}
    node['label'] = label
    fragment = {'label': label,
                'essay_id': random.randint(1, 20),
                'tree': get_node(testimony_id, node, is_parent=True)}
    fragment['tree']['label'] = label

    return fragment

In [412]:
def get_node(testimony_id, node, is_parent=False):
    """Generate a parent or leaf node for a tree structure."""
    if is_parent:
        return {
            'label': node['label'],
            'testimony_id': random.randint(1, 20),
            'media_index': random.randint(1, 20),
            'media_offset': random.randint(1, 20),
            'start_sentence_index': random.randint(1, 20),
            'end_sentence_index': random.randint(1, 20),
            'children': [], }
    else:
        return {'label': node['label'],
                'testimony_id': node['testimony_id'],
                'media_index': float(node['media_index']),
                'media_offset': float(node['media_offset']),
                'start_sentence_index': float(node['start_sentence_index']),
                'end_sentence_index': float(node['end_sentence_index']),
                'children': [], }

In [413]:
def check_if_main_node_exist(node):
    results = mongo.fragments.find({'label':node},{'_id': 0})
    if len(results[0])==0:
        return False
    else:
        return True

In [414]:
def add_main_node(label):
    mongo.fragments.insert(create_parent_node(label))

In [415]:
def delete_main_node(label):
    mongo.fragments.delete_one({'label':label})

In [416]:
def add_testimonial_fragments(fragments):
    if check_if_main_node_exist(fragments['main_node']):
        results = mongo.fragments.find({'label':fragments['main_node']},{'_id':0})[0]
        mid_nodes = [element['label'] for element in results['tree']['children']]
        if fragments['mid_node'] in mid_nodes:
            print ("mid node exists cannot be added")
        else:
            
            mid_node = get_node('r',{'label':fragments['mid_node']},is_parent=True)
            for fragment in fragments['fragments']:
                leaf = get_node(fragment['testimony_id'],fragment)
                mid_node['children'].append(leaf)
            results['tree']['children'].append(mid_node)
            mongo.fragments.replace_one({'label':fragments['main_node']},results)

### Add the main node

In [417]:
main_node = "collapse"
#delete_main_node(main_node)
add_main_node(main_node)

/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


### Set up the query

In [418]:
query = '[lemma="collapse"]'

result = topic_concordancer.main(query,window=10,topicn=25)

### Print the key topics

for i,element in enumerate(result['topic_documents']):
    print (i)
    topic_words =  element['topic_words'][1]
    print (topic_words)
    print ('\n')

### Analyze documents

i=5
for text in result['topic_documents'][i]['texts'][0:25]:
    print (text['matched_text_words'])
    print ('\n')

## Testimonial fragments

### 1.  my whole world

In [419]:
lemmas = ["world","collapse"]

In [420]:
query = create_contextual_query(lemmas,context_length=50)
print (query)

([lemma="world"][]{0,50}[lemma="collapse"])|([lemma="collapse"][]{0,50}[lemma="world"])


In [421]:
domain_term = "my whole world"

In [422]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [423]:
fragment_1 = {}
fragment_1['original_sentence'] = "And, um, I felt and I realized or recognized the fact that is-- this is my world is collapsing."
fragment_1['label']="And, um, I felt and I realized or recognized the fact that is-- this is my world is collapsing."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%5D+%5B%22um%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22felt%22%5D+%5B%22and%22%5D+%5B%22I%22%5D+%5B%22realized%22%5D+%5B%22or%22%5D+%5B%22recognized%22%5D+%5B%22the%22%5D+%5B%22fact%22%5D+%5B%22that%22%5D+%5B%22is%22%5D+%5B%5D%7B0%2C3%7D+%5B%22this%22%5D+%5B%22is%22%5D+%5B%22my%22%5D+%5B%22world%22%5D+%5B%22is%22%5D+%5B%22collapsing%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And , um , I felt and I realized or recognized the fact that is -- this is my world is collapsing . ', 'right': '', 'complete_match': 'And , um , I felt and I realized or recognized the fact that is -- this is my world is collapsing . ', 'testimony_id': 'HVT-11', 'shelfmark': ['Fortunoff HVT-11'], 'token_start': 558, 'token_end': 581}


In [424]:
fragment_2 = {}
fragment_2['original_sentence'] = "So once he said this, we all felt that the whole world collapsed you know around us because there was no really "
fragment_2['label']="we all felt that the whole world collapsed"
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22So%22%5D+%5B%22once%22%5D+%5B%22he%22%5D+%5B%22said%22%5D+%5B%22this%22%5D+%5B%5D+%5B%22we%22%5D+%5B%22all%22%5D+%5B%22felt%22%5D+%5B%22that%22%5D+%5B%22the%22%5D+%5B%22whole%22%5D+%5B%22world%22%5D+%5B%22collapsed%22%5D+%5B%22you%22%5D+%5B%22know%22%5D+%5B%22around%22%5D+%5B%22us%22%5D+%5B%22because%22%5D+%5B%22there%22%5D+%5B%22was%22%5D+%5B%22no%22%5D+%5B%22really%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'So once he said this , we all felt that the whole world collapsed you know around us because there was no really ', 'right': '', 'complete_match': 'So once he said this , we all felt that the whole world collapsed you know around us because there was no really ', 'testimony_id': 'irn504755', 'shelfmark': ['USHMM RG-50.030*0271'], 'token_start': 13935, 'token_end': 13958}


In [425]:
fragment_3 = {}
fragment_3['original_sentence'] = " Our world collapsed. Our life has changed forever."
fragment_3['label']=" Our world collapsed. Our life has changed forever."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22Our%22%5D+%5B%22world%22%5D+%5B%22collapsed%22%5D+%5B%5D+%5B%22Our%22%5D+%5B%22life%22%5D+%5B%22has%22%5D+%5B%22changed%22%5D+%5B%22forever%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'Our world collapsed . Our life has changed forever . ', 'right': '', 'complete_match': 'Our world collapsed . Our life has changed forever . ', 'testimony_id': 'usc_shoah_151', 'shelfmark': ['USC 151'], 'token_start': 16806, 'token_end': 16816}


In [426]:
fragment_4 = {}
fragment_4['original_sentence'] = "And I felt like my whole world had collapsed, because this man, to me, was going to take my mother and father's place, and now he'd gone."
fragment_4['label']= "And I felt like my whole world had collapsed (..)."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22I%22%5D+%5B%22felt%22%5D+%5B%22like%22%5D+%5B%22my%22%5D+%5B%22whole%22%5D+%5B%22world%22%5D+%5B%22had%22%5D+%5B%22collapsed%22%5D+%5B%5D+%5B%22because%22%5D+%5B%22this%22%5D+%5B%22man%22%5D+%5B%5D+%5B%22to%22%5D+%5B%22me%22%5D+%5B%5D+%5B%22was%22%5D+%5B%22going%22%5D+%5B%22to%22%5D+%5B%22take%22%5D+%5B%22my%22%5D+%5B%22mother%22%5D+%5B%22and%22%5D+%5B%22father%22%5D+%5B%5D+%5B%22place%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22now%22%5D+%5B%22he%22%5D+%5B%22%27d%22%5D+%5B%22gone%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "And I felt like my whole world had collapsed , because this man , to me , was going to take my mother and father 's place , and now he 'd gone . ", 'right': '', 'complete_match': "And I felt like my whole world had collapsed , because this man , to me , was going to take my mother and father 's place , and now he 'd gone . ", 'testi

In [427]:
fragment_5 = {}
fragment_5['original_sentence'] = "[PAUSES FOR 7 SECONDS] Our world collapsed completely."
fragment_5['label']= "Our world collapsed completely"
indices = find_sentence_id(fragment_5['original_sentence'])
fragment_5['start_sentence_index']=indices[0]
fragment_5['end_sentence_index']=indices[1]
fragment_5['media_offset'] = 0
fragment_5['media_index'] = 0
fragment_5['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_5)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%5D%7B0%2C3%7D+%5B%22PAUSES%22%5D+%5B%22FOR%22%5D+%5B%227%22%5D+%5B%22SECONDS%22%5D+%5B%5D%7B0%2C3%7D+%5B%22Our%22%5D+%5B%22world%22%5D+%5B%22collapsed%22%5D+%5B%22completely%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'arms . [ PAUSES FOR 7 SECONDS ] Our world collapsed completely . ', 'right': '', 'complete_match': 'arms . [ PAUSES FOR 7 SECONDS ] Our world collapsed completely . ', 'testimony_id': 'usc_shoah_27347', 'shelfmark': ['USC 27347'], 'token_start': 4570, 'token_end': 4583}


In [428]:
add_testimonial_fragments(fragments)

### 2.  die

In [429]:
lemmas = ["die","collapse"]

In [430]:
query = create_contextual_query(lemmas,context_length=25)
print (query)

([lemma="die"][]{0,25}[lemma="collapse"])|([lemma="collapse"][]{0,25}[lemma="die"])


In [431]:
domain_term = "die"

In [432]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [433]:
fragment_1 = {}
fragment_1['original_sentence'] = "And when we went back, people collapsed, died."
fragment_1['label']="And when we went back, people collapsed, died."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22when%22%5D+%5B%22we%22%5D+%5B%22went%22%5D+%5B%22back%22%5D+%5B%5D+%5B%22people%22%5D+%5B%22collapsed%22%5D+%5B%5D+%5B%22died%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And when we went back , people collapsed , died . ', 'right': '', 'complete_match': 'And when we went back , people collapsed , died . ', 'testimony_id': 'HVT-160', 'shelfmark': ['Fortunoff HVT-160'], 'token_start': 4690, 'token_end': 4701}


In [434]:
fragment_2 = {}
fragment_2['original_sentence'] = "than she collapsed, I guess, cause what we saw is they were dead. And she died, and the baby outlived the mother."
fragment_2['label']="(..) than she collapsed (..) And she died, and the baby outlived the mother."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22than%22%5D+%5B%22she%22%5D+%5B%22collapsed%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22guess%22%5D+%5B%5D+%5B%22cause%22%5D+%5B%22what%22%5D+%5B%22we%22%5D+%5B%22saw%22%5D+%5B%22is%22%5D+%5B%22they%22%5D+%5B%22were%22%5D+%5B%22dead%22%5D+%5B%5D+%5B%22And%22%5D+%5B%22she%22%5D+%5B%22died%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22the%22%5D+%5B%22baby%22%5D+%5B%22outlived%22%5D+%5B%22the%22%5D+%5B%22mother%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'than she collapsed , I guess , cause what we saw is they were dead . And she died , and the baby outlived the mother . ', 'right': '', 'complete_match': 'than she collapsed , I guess , cause what we saw is they were dead . And she died , and the baby outlived the mother . ', 'testimony_id': 'irn41517', 'shelfmark': ['USHMM RG-50.030*0567'], 'token_start': 7300, 'token_end': 7327}


In [435]:
fragment_3 = {}
fragment_3['original_sentence'] = "A lot of them died on the steps from the hitting. They collapsed."
fragment_3['label']="A lot of them died on the steps from the hitting. They collapsed."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22A%22%5D+%5B%22lot%22%5D+%5B%22of%22%5D+%5B%22them%22%5D+%5B%22died%22%5D+%5B%22on%22%5D+%5B%22the%22%5D+%5B%22steps%22%5D+%5B%22from%22%5D+%5B%22the%22%5D+%5B%22hitting%22%5D+%5B%5D+%5B%22They%22%5D+%5B%22collapsed%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'A lot of them died on the steps from the hitting . They collapsed . ', 'right': '', 'complete_match': 'A lot of them died on the steps from the hitting . They collapsed . ', 'testimony_id': 'irn504889', 'shelfmark': ['USHMM RG-50.030*0424'], 'token_start': 14271, 'token_end': 14286}


In [436]:
fragment_4 = {}
fragment_4['original_sentence'] = "What actually happened during the death march from Auschwitz, one of the people who died collapsed on the road."
fragment_4['label']= "What actually happened during the death march (..), one of the people who died collapsed on the road."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22What%22%5D+%5B%22actually%22%5D+%5B%22happened%22%5D+%5B%22during%22%5D+%5B%22the%22%5D+%5B%22death%22%5D+%5B%22march%22%5D+%5B%22from%22%5D+%5B%22Auschwitz%22%5D+%5B%5D+%5B%22one%22%5D+%5B%22of%22%5D+%5B%22the%22%5D+%5B%22people%22%5D+%5B%22who%22%5D+%5B%22died%22%5D+%5B%22collapsed%22%5D+%5B%22on%22%5D+%5B%22the%22%5D+%5B%22road%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'What actually happened during the death march from Auschwitz , one of the people who died collapsed on the road . ', 'right': '', 'complete_match': 'What actually happened during the death march from Auschwitz , one of the people who died collapsed on the road . ', 'testimony_id': 'usc_shoah_5758', 'shelfmark': ['USC 5758'], 'token_start': 27189, 'token_end': 27210}


In [437]:
add_testimonial_fragments(fragments)

### 3.  shoot

In [438]:
lemmas = ["shoot","collapse"]

In [439]:
query = create_contextual_query(lemmas,context_length=25)
print (query)

([lemma="shoot"][]{0,25}[lemma="collapse"])|([lemma="collapse"][]{0,25}[lemma="shoot"])


In [440]:
domain_term = "shoot"

In [441]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [442]:
fragment_1 = {}
fragment_1['original_sentence'] = "And they collapsed, even in Theresienstadt, they would shoot them on the spot. "
fragment_1['label']="And they collapsed, (..) they would shoot them on the spot. "
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22they%22%5D+%5B%22collapsed%22%5D+%5B%5D+%5B%22even%22%5D+%5B%22in%22%5D+%5B%22Theresienstadt%22%5D+%5B%5D+%5B%22they%22%5D+%5B%22would%22%5D+%5B%22shoot%22%5D+%5B%22them%22%5D+%5B%22on%22%5D+%5B%22the%22%5D+%5B%22spot%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And they collapsed , even in Theresienstadt , they would shoot them on the spot . ', 'right': '', 'complete_match': 'And they collapsed , even in Theresienstadt , they would shoot them on the spot . ', 'testimony_id': 'HVT-66', 'shelfmark': ['Fortunoff HVT-66'], 'token_start': 9352, 'token_end': 9368}


In [443]:
fragment_2 = {}
fragment_2['original_sentence'] = "people collapsed left, right and center, and those who collapsed were shot in the head by the SS there and then"
fragment_2['label']="(..)people collapsed left, right and center, and those who collapsed were shot in the head by the SS there and then (..)"
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22people%22%5D+%5B%22collapsed%22%5D+%5B%22left%22%5D+%5B%5D+%5B%22right%22%5D+%5B%22and%22%5D+%5B%22center%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22those%22%5D+%5B%22who%22%5D+%5B%22collapsed%22%5D+%5B%22were%22%5D+%5B%22shot%22%5D+%5B%22in%22%5D+%5B%22the%22%5D+%5B%22head%22%5D+%5B%22by%22%5D+%5B%22the%22%5D+%5B%22SS%22%5D+%5B%22there%22%5D+%5B%22and%22%5D+%5B%22then%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'people collapsed left , right and center , and those who collapsed were shot in the head by the SS there and then ', 'right': '', 'complete_match': 'people collapsed left , right and center , and those who collapsed were shot in the head by the SS there and then ', 'testimony_id': 'irn518264', 'shelfmark': ['USHMM RG-50.562*0010'], 'token_start': 15499, 'token_end': 15522}


In [444]:
fragment_3 = {}
fragment_3['original_sentence'] = "But again, SS shot them. And when people collapsed, they were shot."
fragment_3['label']="But again, SS shot them. And when people collapsed, they were shot."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22But%22%5D+%5B%22again%22%5D+%5B%5D+%5B%22SS%22%5D+%5B%22shot%22%5D+%5B%22them%22%5D+%5B%5D+%5B%22And%22%5D+%5B%22when%22%5D+%5B%22people%22%5D+%5B%22collapsed%22%5D+%5B%5D+%5B%22they%22%5D+%5B%22were%22%5D+%5B%22shot%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'But again , SS shot them . And when people collapsed , they were shot . ', 'right': '', 'complete_match': 'But again , SS shot them . And when people collapsed , they were shot . ', 'testimony_id': 'usc_shoah_5758', 'shelfmark': ['USC 5758'], 'token_start': 19944, 'token_end': 19960}


In [445]:
fragment_4 = {}
fragment_4['original_sentence'] = "And during this death march, anybody who collapsed, anybody-- and there were very many-- they were shot on the spot and left there."
fragment_4['label']= "anybody who collapsed (..) they were shot on the spot and left there."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22during%22%5D+%5B%22this%22%5D+%5B%22death%22%5D+%5B%22march%22%5D+%5B%5D+%5B%22anybody%22%5D+%5B%22who%22%5D+%5B%22collapsed%22%5D+%5B%5D+%5B%22anybody%22%5D+%5B%5D%7B0%2C3%7D+%5B%22and%22%5D+%5B%22there%22%5D+%5B%22were%22%5D+%5B%22very%22%5D+%5B%22many%22%5D+%5B%5D%7B0%2C3%7D+%5B%22they%22%5D+%5B%22were%22%5D+%5B%22shot%22%5D+%5B%22on%22%5D+%5B%22the%22%5D+%5B%22spot%22%5D+%5B%22and%22%5D+%5B%22left%22%5D+%5B%22there%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And during this death march , anybody who collapsed , anybody -- and there were very many -- they were shot on the spot and left there . ', 'right': '', 'complete_match': 'And during this death march , anybody who collapsed , anybody -- and there were very many -- they were shot on the spot and left there . ', 'testimony_id': 'usc_shoah_7093', 'shelfmark': ['USC 7093'], 'token_start': 

In [446]:
fragment_5 = {}
fragment_5['original_sentence'] = "you said, I have to sit down to lie down, the SS shot you."
fragment_5['label']= "Because the minute that you collapse, or (..) you said, I have to sit down to lie down, the SS shot you."
indices = find_sentence_id(fragment_5['original_sentence'])
fragment_5['start_sentence_index']=indices[0]
fragment_5['end_sentence_index']=indices[1]
fragment_5['media_offset'] = 0
fragment_5['media_index'] = 0
fragment_5['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_5)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22you%22%5D+%5B%22said%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22have%22%5D+%5B%22to%22%5D+%5B%22sit%22%5D+%5B%22down%22%5D+%5B%22to%22%5D+%5B%22lie%22%5D+%5B%22down%22%5D+%5B%5D+%5B%22the%22%5D+%5B%22SS%22%5D+%5B%22shot%22%5D+%5B%22you%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'you said , I have to sit down to lie down , the SS shot you . ', 'right': '', 'complete_match': 'you said , I have to sit down to lie down , the SS shot you . ', 'testimony_id': 'usc_shoah_17135', 'shelfmark': ['USC 17135'], 'token_start': 14856, 'token_end': 14873}


In [447]:
add_testimonial_fragments(fragments)

### 4.  Leave

In [448]:
lemmas = ["leave","collapse"]

In [449]:
query = create_contextual_query(lemmas,context_length=40)
print (query)

([lemma="leave"][]{0,40}[lemma="collapse"])|([lemma="collapse"][]{0,40}[lemma="leave"])


In [450]:
domain_term = "leave"

In [451]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [452]:
fragment_1 = {}
fragment_1['original_sentence'] = "People just collapsed, and they left them there."
fragment_1['label']="People just collapsed, and they left them there."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22People%22%5D+%5B%22just%22%5D+%5B%22collapsed%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22they%22%5D+%5B%22left%22%5D+%5B%22them%22%5D+%5B%22there%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'People just collapsed , and they left them there . ', 'right': '', 'complete_match': 'People just collapsed , and they left them there . ', 'testimony_id': 'HVT-22', 'shelfmark': ['Fortunoff HVT-22'], 'token_start': 14023, 'token_end': 14033}


In [453]:
fragment_2 = {}
fragment_2['original_sentence'] = "They collapsed, they killed them, and left them."
fragment_2['label']="They collapsed, they killed them, and left them."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22They%22%5D+%5B%22collapsed%22%5D+%5B%5D+%5B%22they%22%5D+%5B%22killed%22%5D+%5B%22them%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22left%22%5D+%5B%22them%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'They collapsed , they killed them , and left them . ', 'right': '', 'complete_match': 'They collapsed , they killed them , and left them . ', 'testimony_id': 'usc_shoah_2938', 'shelfmark': ['USC 2938'], 'token_start': 12352, 'token_end': 12363}


In [454]:
fragment_3 = {}
fragment_3['original_sentence'] = "And during this death march, anybody who collapsed, anybody-- and there were very many-- they were shot on the spot and left there. "
fragment_3['label']="anybody who collapsed (..) they were shot on the spot and left there. "
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22during%22%5D+%5B%22this%22%5D+%5B%22death%22%5D+%5B%22march%22%5D+%5B%5D+%5B%22anybody%22%5D+%5B%22who%22%5D+%5B%22collapsed%22%5D+%5B%5D+%5B%22anybody%22%5D+%5B%5D%7B0%2C3%7D+%5B%22and%22%5D+%5B%22there%22%5D+%5B%22were%22%5D+%5B%22very%22%5D+%5B%22many%22%5D+%5B%5D%7B0%2C3%7D+%5B%22they%22%5D+%5B%22were%22%5D+%5B%22shot%22%5D+%5B%22on%22%5D+%5B%22the%22%5D+%5B%22spot%22%5D+%5B%22and%22%5D+%5B%22left%22%5D+%5B%22there%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And during this death march , anybody who collapsed , anybody -- and there were very many -- they were shot on the spot and left there . ', 'right': '', 'complete_match': 'And during this death march , anybody who collapsed , anybody -- and there were very many -- they were shot on the spot and left there . ', 'testimony_id': 'usc_shoah_7093', 'shelfmark': ['USC 7093'], 'token_start': 

In [455]:
add_testimonial_fragments(fragments)

### 5.  

In [456]:
lemmas = ["work","collapse"]

In [457]:
query = create_contextual_query(lemmas,context_length=25)
print (query)

([lemma="work"][]{0,25}[lemma="collapse"])|([lemma="collapse"][]{0,25}[lemma="work"])


In [458]:
domain_term = "work"

In [459]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [460]:
fragment_1 = {}
fragment_1['original_sentence'] = "It was hard work. I remember one day one of the prisoners collapsed on the ramp with his sack of cement."
fragment_1['label']="It was hard work. (..) one day one of the prisoners collapsed on the ramp with his sack of cement."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22It%22%5D+%5B%22was%22%5D+%5B%22hard%22%5D+%5B%22work%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22remember%22%5D+%5B%22one%22%5D+%5B%22day%22%5D+%5B%22one%22%5D+%5B%22of%22%5D+%5B%22the%22%5D+%5B%22prisoners%22%5D+%5B%22collapsed%22%5D+%5B%22on%22%5D+%5B%22the%22%5D+%5B%22ramp%22%5D+%5B%22with%22%5D+%5B%22his%22%5D+%5B%22sack%22%5D+%5B%22of%22%5D+%5B%22cement%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'It was hard work . I remember one day one of the prisoners collapsed on the ramp with his sack of cement . ', 'right': '', 'complete_match': 'It was hard work . I remember one day one of the prisoners collapsed on the ramp with his sack of cement . ', 'testimony_id': 'irn506634', 'shelfmark': ['USHMM RG-50.106*0122'], 'token_start': 13398, 'token_end': 13421}


In [461]:
fragment_2 = {}
fragment_2['original_sentence'] = "One day, I was sitting after work in the bunk there, in the barrack, and I collapsed."
fragment_2['label']="One day, I was sitting after work in the bunk there, in the barrack, and I collapsed."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22One%22%5D+%5B%22day%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22sitting%22%5D+%5B%22after%22%5D+%5B%22work%22%5D+%5B%22in%22%5D+%5B%22the%22%5D+%5B%22bunk%22%5D+%5B%22there%22%5D+%5B%5D+%5B%22in%22%5D+%5B%22the%22%5D+%5B%22barrack%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22I%22%5D+%5B%22collapsed%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'One day , I was sitting after work in the bunk there , in the barrack , and I collapsed . ', 'right': '', 'complete_match': 'One day , I was sitting after work in the bunk there , in the barrack , and I collapsed . ', 'testimony_id': 'usc_shoah_10710', 'shelfmark': ['USC 10710'], 'token_start': 12778, 'token_end': 12799}


In [462]:
fragment_3 = {}
fragment_3['original_sentence'] = "going to work, come late, and collapsing, [SIGHS] nothing more."
fragment_3['label']="It was ghetto life, going to work, come late, and collapsing, nothing more."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22going%22%5D+%5B%22to%22%5D+%5B%22work%22%5D+%5B%5D+%5B%22come%22%5D+%5B%22late%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22collapsing%22%5D+%5B%5D+%5B%5D%7B0%2C3%7D+%5B%22SIGHS%22%5D+%5B%5D%7B0%2C3%7D+%5B%22nothing%22%5D+%5B%22more%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'going to work , come late , and collapsing , [ SIGHS ] nothing more . ', 'right': '', 'complete_match': 'going to work , come late , and collapsing , [ SIGHS ] nothing more . ', 'testimony_id': 'usc_shoah_13063', 'shelfmark': ['USC 13063'], 'token_start': 15905, 'token_end': 15921}


In [463]:
fragment_4 = {}
fragment_4['original_sentence'] = "The third day, I was so sick that I collapsed in the working trenches, where we were working."
fragment_4['label']= "(..) I was so sick that I collapsed in the working trenches, where we were working."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22The%22%5D+%5B%22third%22%5D+%5B%22day%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22so%22%5D+%5B%22sick%22%5D+%5B%22that%22%5D+%5B%22I%22%5D+%5B%22collapsed%22%5D+%5B%22in%22%5D+%5B%22the%22%5D+%5B%22working%22%5D+%5B%22trenches%22%5D+%5B%5D+%5B%22where%22%5D+%5B%22we%22%5D+%5B%22were%22%5D+%5B%22working%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'The third day , I was so sick that I collapsed in the working trenches , where we were working . ', 'right': '', 'complete_match': 'The third day , I was so sick that I collapsed in the working trenches , where we were working . ', 'testimony_id': 'usc_shoah_3341', 'shelfmark': ['USC 3341'], 'token_start': 9848, 'token_end': 9869}


In [464]:
fragment_5 = {}
fragment_5['original_sentence'] = " I mean, we collapsed. It was very difficult work for women."
fragment_5['label']= "I mean, we collapsed. It was very difficult work for women."
indices = find_sentence_id(fragment_5['original_sentence'])
fragment_5['start_sentence_index']=indices[0]
fragment_5['end_sentence_index']=indices[1]
fragment_5['media_offset'] = 0
fragment_5['media_index'] = 0
fragment_5['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_5)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22I%22%5D+%5B%22mean%22%5D+%5B%5D+%5B%22we%22%5D+%5B%22collapsed%22%5D+%5B%5D+%5B%22It%22%5D+%5B%22was%22%5D+%5B%22very%22%5D+%5B%22difficult%22%5D+%5B%22work%22%5D+%5B%22for%22%5D+%5B%22women%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'I mean , we collapsed . It was very difficult work for women . ', 'right': '', 'complete_match': 'I mean , we collapsed . It was very difficult work for women . ', 'testimony_id': 'HVT-150', 'shelfmark': ['Fortunoff HVT-150'], 'token_start': 13527, 'token_end': 13541}


In [465]:
add_testimonial_fragments(fragments)

### 6. stand 

In [466]:
lemmas = ['stand','collapse']

In [467]:
query = create_contextual_query(lemmas,context_length=25)
print (query)

([lemma="stand"][]{0,25}[lemma="collapse"])|([lemma="collapse"][]{0,25}[lemma="stand"])


In [468]:
domain_term = "stand"

In [469]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [470]:
fragment_1 = {}
fragment_1['original_sentence'] = " You stood there for 16, 12 hours. That depends. We always left a few of them out there. And when we went back, people collapsed, died."
fragment_1['label']="You stood there for 16, 12 hours. That depends. (..) And when we went back, people collapsed, died."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22You%22%5D+%5B%22stood%22%5D+%5B%22there%22%5D+%5B%22for%22%5D+%5B%2216%22%5D+%5B%5D+%5B%2212%22%5D+%5B%22hours%22%5D+%5B%5D+%5B%22That%22%5D+%5B%22depends%22%5D+%5B%5D+%5B%22We%22%5D+%5B%22always%22%5D+%5B%22left%22%5D+%5B%22a%22%5D+%5B%22few%22%5D+%5B%22of%22%5D+%5B%22them%22%5D+%5B%22out%22%5D+%5B%22there%22%5D+%5B%5D+%5B%22And%22%5D+%5B%22when%22%5D+%5B%22we%22%5D+%5B%22went%22%5D+%5B%22back%22%5D+%5B%5D+%5B%22people%22%5D+%5B%22collapsed%22%5D+%5B%5D+%5B%22died%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'You stood there for 16 , 12 hours . That depends . We always left a few of them out there . And when we went back , people collapsed , died . ', 'right': '', 'complete_match': 'You stood there for 16 , 12 hours . That depends . We always left a few of them out there . And when we went back , people collapsed , died . ', 'testimony_id': 'HVT-160', 'shelfma

In [471]:
fragment_2 = {}
fragment_2['original_sentence'] = "And then the next thing, the elderly couldn't stand on their feet. They started collapsing. "
fragment_2['label']="And then the next thing, the elderly couldn't stand on their feet. They started collapsing."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22then%22%5D+%5B%22the%22%5D+%5B%22next%22%5D+%5B%22thing%22%5D+%5B%5D+%5B%22the%22%5D+%5B%22elderly%22%5D+%5B%22could%22%5D+%5B%5D+%5B%22stand%22%5D+%5B%22on%22%5D+%5B%22their%22%5D+%5B%22feet%22%5D+%5B%5D+%5B%22They%22%5D+%5B%22started%22%5D+%5B%22collapsing%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "And then the next thing , the elderly could n't stand on their feet . They started collapsing . ", 'right': '', 'complete_match': "And then the next thing , the elderly could n't stand on their feet . They started collapsing . ", 'testimony_id': 'usc_shoah_11272', 'shelfmark': ['USC 11272'], 'token_start': 5730, 'token_end': 5749}


In [472]:
fragment_3 = {}
fragment_3['original_sentence'] = "And the next day, they took us; and we been standing in the freezing cold. And some girls collapsed."
fragment_3['label']="(..) we been standing in the freezing cold. And some girls collapsed."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22the%22%5D+%5B%22next%22%5D+%5B%22day%22%5D+%5B%5D+%5B%22they%22%5D+%5B%22took%22%5D+%5B%22us%22%5D+%5B%22%3B%22%5D+%5B%22and%22%5D+%5B%22we%22%5D+%5B%22been%22%5D+%5B%22standing%22%5D+%5B%22in%22%5D+%5B%22the%22%5D+%5B%22freezing%22%5D+%5B%22cold%22%5D+%5B%5D+%5B%22And%22%5D+%5B%22some%22%5D+%5B%22girls%22%5D+%5B%22collapsed%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And the next day , they took us ; and we been standing in the freezing cold . And some girls collapsed . ', 'right': '', 'complete_match': 'And the next day , they took us ; and we been standing in the freezing cold . And some girls collapsed . ', 'testimony_id': 'irn504535', 'shelfmark': ['USHMM RG-50.030*0030'], 'token_start': 4591, 'token_end': 4614}


In [473]:
fragment_4 = {}
fragment_4['original_sentence'] = "I'm standing in line and it's muddy and it's cold. This happened May the 1st, May the 2nd. And I collapse, I black out."
fragment_4['label']= "I'm standing in line and it's muddy and it's cold. (..) And I collapse, I black out."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22I%22%5D+%5B%22%27m%22%5D+%5B%22standing%22%5D+%5B%22in%22%5D+%5B%22line%22%5D+%5B%22and%22%5D+%5B%22it%22%5D+%5B%5D+%5B%22muddy%22%5D+%5B%22and%22%5D+%5B%22it%22%5D+%5B%5D+%5B%22cold%22%5D+%5B%5D+%5B%22This%22%5D+%5B%22happened%22%5D+%5B%22May%22%5D+%5B%22the%22%5D+%5B%221st%22%5D+%5B%5D+%5B%22May%22%5D+%5B%22the%22%5D+%5B%222nd%22%5D+%5B%5D+%5B%22And%22%5D+%5B%22I%22%5D+%5B%22collapse%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22black%22%5D+%5B%22out%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "I 'm standing in line and it 's muddy and it 's cold . This happened May the 1st , May the 2nd . And I collapse , I black out . ", 'right': '', 'complete_match': "I 'm standing in line and it 's muddy and it 's cold . This happened May the 1st , May the 2nd . And I collapse , I black out . ", 'testimony_id': 'usc_shoah_17526', 'shelfmark': ['USC 17526'], 'token_start': 15272, 'tok

In [474]:
fragment_5 = {}
fragment_5['original_sentence'] = "And then they let us stand. And lot of girls collapse, fainted."
fragment_5['label']= "And then they let us stand. And lot of girls collapse, fainted."
indices = find_sentence_id(fragment_5['original_sentence'])
fragment_5['start_sentence_index']=indices[0]
fragment_5['end_sentence_index']=indices[1]
fragment_5['media_offset'] = 0
fragment_5['media_index'] = 0
fragment_5['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_5)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22then%22%5D+%5B%22they%22%5D+%5B%22let%22%5D+%5B%22us%22%5D+%5B%22stand%22%5D+%5B%5D+%5B%22And%22%5D+%5B%22lot%22%5D+%5B%22of%22%5D+%5B%22girls%22%5D+%5B%22collapse%22%5D+%5B%5D+%5B%22fainted%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And then they let us stand . And lot of girls collapse , fainted . ', 'right': '', 'complete_match': 'And then they let us stand . And lot of girls collapse , fainted . ', 'testimony_id': 'usc_shoah_19441', 'shelfmark': ['USC 19441'], 'token_start': 7016, 'token_end': 7031}


In [475]:
add_testimonial_fragments(fragments)

### 6.  Hit

In [476]:
lemmas = ["hit","collapse"]

In [477]:
query = create_contextual_query(lemmas,context_length=50)
print (query)

([lemma="hit"][]{0,50}[lemma="collapse"])|([lemma="collapse"][]{0,50}[lemma="hit"])


In [478]:
domain_term = "hit"

In [479]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [480]:
fragment_1 = {}
fragment_1['original_sentence'] = "[PAUSES FOR 3 SECONDS] And he hit this inmate with his rifle butt in his face. The guy"
fragment_1['label']="And he hit this inmate with his rifle butt in his face. The (..) guy collapsed"
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%5D%7B0%2C3%7D+%5B%22PAUSES%22%5D+%5B%22FOR%22%5D+%5B%223%22%5D+%5B%22SECONDS%22%5D+%5B%5D%7B0%2C3%7D+%5B%22And%22%5D+%5B%22he%22%5D+%5B%22hit%22%5D+%5B%22this%22%5D+%5B%22inmate%22%5D+%5B%22with%22%5D+%5B%22his%22%5D+%5B%22rifle%22%5D+%5B%22butt%22%5D+%5B%22in%22%5D+%5B%22his%22%5D+%5B%22face%22%5D+%5B%5D+%5B%22The%22%5D+%5B%22guy%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'stop . [ PAUSES FOR 3 SECONDS ] And he hit this inmate with his rifle butt in his face . The guy ', 'right': '', 'complete_match': 'stop . [ PAUSES FOR 3 SECONDS ] And he hit this inmate with his rifle butt in his face . The guy ', 'testimony_id': 'usc_shoah_12413', 'shelfmark': ['USC 12413'], 'token_start': 9699, 'token_end': 9722}


In [481]:
fragment_2 = {}
fragment_2['original_sentence'] = "And I see the SS guard hitting him, and hitting him, and hitting him. And he collapses."
fragment_2['label']="And I see the SS guard hitting him, and hitting him, and hitting him. And he collapses."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22I%22%5D+%5B%22see%22%5D+%5B%22the%22%5D+%5B%22SS%22%5D+%5B%22guard%22%5D+%5B%22hitting%22%5D+%5B%22him%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22hitting%22%5D+%5B%22him%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22hitting%22%5D+%5B%22him%22%5D+%5B%5D+%5B%22And%22%5D+%5B%22he%22%5D+%5B%22collapses%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And I see the SS guard hitting him , and hitting him , and hitting him . And he collapses . ', 'right': '', 'complete_match': 'And I see the SS guard hitting him , and hitting him , and hitting him . And he collapses . ', 'testimony_id': 'usc_shoah_8947', 'shelfmark': ['USC 8947'], 'token_start': 17574, 'token_end': 17595}


In [482]:
fragment_3 = {}
fragment_3['original_sentence'] = "And suddenly I saw this German soldier with his rifle butt, you know, hitting that man over the head. And the man collapsed, cursing at him."
fragment_3['label']="(..) I saw this German soldier with his rifle butt, you know, hitting that man over the head. And the man collapsed (..)"
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22suddenly%22%5D+%5B%22I%22%5D+%5B%22saw%22%5D+%5B%22this%22%5D+%5B%22German%22%5D+%5B%22soldier%22%5D+%5B%22with%22%5D+%5B%22his%22%5D+%5B%22rifle%22%5D+%5B%22butt%22%5D+%5B%5D+%5B%22you%22%5D+%5B%22know%22%5D+%5B%5D+%5B%22hitting%22%5D+%5B%22that%22%5D+%5B%22man%22%5D+%5B%22over%22%5D+%5B%22the%22%5D+%5B%22head%22%5D+%5B%5D+%5B%22And%22%5D+%5B%22the%22%5D+%5B%22man%22%5D+%5B%22collapsed%22%5D+%5B%5D+%5B%22cursing%22%5D+%5B%22at%22%5D+%5B%22him%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And suddenly I saw this German soldier with his rifle butt , you know , hitting that man over the head . And the man collapsed , cursing at him . ', 'right': '', 'complete_match': 'And suddenly I saw this German soldier with his rifle butt , you know , hitting that man over the head . And the man collapsed , cursing at him . ', 'testimony_id': 'usc_shoah_10406'

In [483]:
add_testimonial_fragments(fragments)